[View in Colaboratory](https://colab.research.google.com/github/tonychen233/colab_tensorflow/blob/master/cnn_simpsons.ipynb)

**Connect to Google Drive**

In [1]:
# 授权绑定Google Drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 22280 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.0-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleus

In [0]:
# 指定Google Drive云端硬盘的根目录，名为drive
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
# 指定当前的工作目录
import os

# 此处为google drive中的文件路径,drive为之前指定的工作根目录，要加上
os.chdir("drive/Project/Data/CNN_Simpsons/") 

In [4]:
!ls


predict  test_set  training_set


**Importing libraries**

In [5]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from timeit import default_timer as timer
from keras.preprocessing import image
import numpy as np
import os

Using TensorFlow backend.


**Setting Image size**

Resized all the images to 128x128. Can choose a size that's greater or lower depending upon the computational power of the system.

In [0]:
image_height = 128
image_width = 128

**Constructing the neural network**

A CNN consists of an input and an output layer, as well as multiple hidden layers. I constructed 4 hidden layers for my neural network.

Convolutional layers apply a convolution operation to the input, passing the result to the next layer. The convolution emulates the response of an individual neuron to visual stimuli. Each convolutional neuron processes data only for its receptive field.

Max pooling uses the maximum value from each of a cluster of neurons at the prior layer, greatly reducing the size of the image after feature extraction.

In [0]:
# Initialising the CNN
predator = Sequential()
# Step 1 - Convolution
predator.add(Conv2D(64, (3, 3), activation="relu", input_shape=(image_height, image_width, 3)))
# Step 2 - Pooling
predator.add(MaxPooling2D(pool_size = (2, 2)))
# Adding a second convolutional layer
predator.add(Conv2D(128, (3, 3), activation="relu"))
predator.add(MaxPooling2D(pool_size = (2, 2)))
# Adding a third convolutional layer
predator.add(Conv2D(256, (3, 3), activation="relu"))
predator.add(MaxPooling2D(pool_size = (2, 2)))
# Adding a fourth convolutional layer
predator.add(Conv2D(128, (3, 3), activation="relu"))
predator.add(MaxPooling2D(pool_size = (2, 2)))
# Adding a fifth convolutional layer
predator.add(Conv2D(64, (3, 3), activation="relu"))
predator.add(MaxPooling2D(pool_size = (2, 2)))
# Step 3 - Flattening
predator.add(Flatten())    #flattens the 3D image array to a single row array
# Step 4 - Full connection
predator.add(Dense(units=32, activation="relu"))
predator.add(Dense(units=10, activation="softmax"))   #output layer with 10 neurons.. each corresponding to a character
# Compiling the CNN
predator.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

**Defining the training set and test set**

In [8]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#ImageDataGenerator augments images, creating multiple versions of the same image
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('training_set',
                                                 target_size = (image_height, image_width),
                                                 batch_size = 50,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('test_set',
                                            target_size = (image_height, image_width),
                                            batch_size = 50,
                                            class_mode = 'categorical')

Found 8000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


**Fitting the CNN to the images**

This starts the training process. It took me close to 10 minutes for running 100 epochs to return an accuracy of more than 90 percent.

In [9]:
predator.fit_generator(training_set,
                         steps_per_epoch = 16,
                         epochs = 100,
                         validation_data = test_set,
                         validation_steps = 4)

Epoch 1/100
16/16 [==============================] - 639s 40s/step - loss: 2.3078 - acc: 0.0912 - val_loss: 2.3008 - val_acc: 0.1300
Epoch 2/100
16/16 [==============================] - 384s 24s/step - loss: 2.3019 - acc: 0.1200 - val_loss: 2.3004 - val_acc: 0.1200
Epoch 3/100
16/16 [==============================] - 503s 31s/step - loss: 2.2612 - acc: 0.1412 - val_loss: 2.1784 - val_acc: 0.1750
Epoch 4/100
16/16 [==============================] - 511s 32s/step - loss: 2.1692 - acc: 0.1975 - val_loss: 2.0606 - val_acc: 0.2950
Epoch 5/100
16/16 [==============================] - 521s 33s/step - loss: 2.0860 - acc: 0.2675 - val_loss: 2.0234 - val_acc: 0.2950
Epoch 6/100
16/16 [==============================] - 509s 32s/step - loss: 2.0067 - acc: 0.3088 - val_loss: 2.0545 - val_acc: 0.2150
Epoch 7/100
16/16 [==============================] - 508s 32s/step - loss: 1.9460 - acc: 0.3162 - val_loss: 1.8731 - val_acc: 0.3400
Epoch 8/100
16/16 [==============================] - 517s 32s/step - 

**Prediction on a new picture**

The dataset also has an independent folder, predict, containing close to 500 images of the 10 characters.

In [0]:
# prediction on a new picture 
result = []
path = 'predict'
files = os.listdir(path)

for file in files:
    test_image = image.load_img('predict/'+file, target_size=(image_height, image_width))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)

    pred = predator.predict_on_batch(test_image)
    result.append(pred)

result = np.asarray(result)

**Generating a csv file with the results**

In [0]:
index = files
import pandas as pd
predictions = result[:, [0]][:,0]
df = pd.DataFrame(index=index)

df['bart'] = predictions[:,0]
df['charles'] = predictions[:,1]
df['homer'] = predictions[:,2]
df['krusty'] = predictions[:,3]
df['lisa'] = predictions[:,4]
df['marge'] = predictions[:,5]
df['milhouse'] = predictions[:,6]
df['moe'] = predictions[:,7]
df['ned'] = predictions[:,8]
df['principal'] = predictions[:,9]

df = df.astype(int)

df.to_csv('predict/predictions.csv')

**Save the trained model**

In [0]:
#save weight

predator.save_weights('predict/simpsons-CNN_V1.hdf5')